<a href="https://colab.research.google.com/github/Muhammad-Usama-07/agentic_ai/blob/main/Different_Basic_Agents.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Packages

In [2]:
!pip install -U langchain langchain-openai python-dotenv langchain-groq
!pip install -U langchain langchain-core langchain-community langchain-groq
!pip install -U langchain langchain-core langchain-community langchain-groq langchain-classic

# Agents Description

## Agent 1
- Take question from the user
- Give answer by using a tool(a method)

# Agent 1 (Langchain + OpenAI)

#### set key

In [ ]:
import os
os.environ["OPENAI_API_KEY"] = ""


In [3]:
print('aaa')

aaa


In [ ]:
from langchain_openai import ChatOpenAI
from langchain.tools import tool
from langchain.agents import create_openai_functions_agent, AgentExecutor
from langchain.prompts import ChatPromptTemplate
from dotenv import load_dotenv

load_dotenv()

# Define the tool
@tool
def get_weather(city: str) -> str:
    """Get weather for a given city."""
    return f"It's always sunny in {city}!"

# LLM
llm = ChatOpenAI(model="gpt-4o", temperature=0)

# Prompt
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant"),
    ("human", "{input}"),
    ("placeholder", "{agent_scratchpad}")
])

# Create agent
agent = create_openai_functions_agent(
    llm=llm,
    tools=[get_weather],
    prompt=prompt
)

# Agent executor
agent_executor = AgentExecutor(
    agent=agent,
    tools=[get_weather],
    verbose=True
)

# Run
result = agent_executor.invoke({
    "input": "What is the weather in San Francisco?"
})

print(result["output"])


# Agent 1 (Langchain + Groq)

In [1]:
import os

os.environ["GROQ_API_KEY"] = ""


In [16]:
from langchain_groq import ChatGroq
from langchain.tools import tool
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.messages import ToolMessage

#
# ------------------------- TOOL -------------------------
@tool
def get_code() -> str:
    """Returns a secret code unknown to the model."""
    print("TOOL EXECUTED: get_code")
    return "cod-12345"

tools = [get_code]

# ------------------------- LLM -------------------------
llm = ChatGroq(
    model="llama-3.3-70b-versatile",
    temperature=0
)

llm_with_tools = llm.bind_tools(tools)

# ------------------------- PROMPT -------------------------
prompt = ChatPromptTemplate.from_messages([
    (
        "system",
        "You do NOT know the secret code. "
        "The ONLY way to get it is by calling the provided tool. "
        "You MUST use the tool to answer."
    ),
    ("human", "{input}")
])

# ------------------------- USER QUESTION -------------------------
messages = prompt.invoke(
    {"input": "What is the secret code?"}
).to_messages()

# ------------------------- STEP 1 (REQUEST TOOL) -------------------------
response = llm_with_tools.invoke(messages)

print("\nModel response:")
print(response)

# ------------------------- TOOL EXECUTION -------------------------
if response.tool_calls:
    tool_call = response.tool_calls[0]

    print("\nTool requested by model:")
    print(tool_call)

    # Execute tool
    tool_result = get_code.invoke(tool_call["args"])

    # ------------------------- STEP 2 (FINAL ANSWER) -------------------------
    final_response = llm.invoke([
        *messages,
        response,
        ToolMessage(
            content=tool_result,
            tool_call_id=tool_call["id"]
        )
    ])

    print("\nFINAL ANSWER:")
    print(final_response.content)
else:
    print("Model did NOT call the tool")



Model response:
content='' additional_kwargs={'tool_calls': [{'id': 'ymz3y8sag', 'function': {'arguments': '{}', 'name': 'get_code'}, 'type': 'function'}]} response_metadata={'token_usage': {'completion_tokens': 8, 'prompt_tokens': 242, 'total_tokens': 250, 'completion_time': 0.015676753, 'completion_tokens_details': None, 'prompt_time': 0.020151408, 'prompt_tokens_details': None, 'queue_time': 0.038662682, 'total_time': 0.035828161}, 'model_name': 'llama-3.3-70b-versatile', 'system_fingerprint': 'fp_bebe2dd4fb', 'service_tier': 'on_demand', 'finish_reason': 'tool_calls', 'logprobs': None, 'model_provider': 'groq'} id='lc_run--019bf4fb-ea3d-75f2-b8f9-a285683d45d7-0' tool_calls=[{'name': 'get_code', 'args': {}, 'id': 'ymz3y8sag', 'type': 'tool_call'}] invalid_tool_calls=[] usage_metadata={'input_tokens': 242, 'output_tokens': 8, 'total_tokens': 250}

Tool requested by model:
{'name': 'get_code', 'args': {}, 'id': 'ymz3y8sag', 'type': 'tool_call'}
TOOL EXECUTED: get_code

FINAL ANSWER:
